In [40]:
M = [[1, 2, 3, 4, 5],
    [2, 4, 8, 16, 32],
    [3, 6, 9, 12, 15],
    [6, 12, 18, 24, 30]]

S = [0, 0, 1, 3]
T = [1, 3, 3, 0]

sum( [sum([M[s][t] for t in T]) for s in S] )

126

In [ ]:
import numpy as np

def density(M, S, T) -> float:
    '''For each row index in T, finds the sum of all values in M at columns in S. 
    Sums those all together, then returns divided by the product of the lists' length'''

    if S is None or T is None:
        return 0
    else:
        return sum( [sum([M[s][t] for t in T]) for s in S] ) / np.sqrt(len(S)*len(T))
    
    
def rowsum(M, u:int, T) -> int:
    '''Sums all elements in M row u, that appear on indices in T'''
    
    return sum([M[u][t] for t in T])


def colsum(M, S, v:int) -> int:
    '''Sums all elements in M column v, that appear on indices in S'''
    
    return sum([M[t][u] for t in T])


def expand(self, M, S, T, u:int, v:int):
    '''
    Appends u to S if that increases the density(M, S, T)
    Appends v to T if that increases the density(M, S, T)
    Returns the S and T with/without the appending
    '''

    S1 = S.copy().append(u)
    T1 = T.copy().append(v)

    d0 = density(M,S,T)
    d1 = density(M,S1,T)
    d2 = density(M,S,T1)

    if d1 > d0 and u not in S:
        S.append(u)
    if d2 > d0 and v not in T:
        T.append(v)

    return S,T


def condense(self, M, S, T):
    '''Keeps removing indices from S and T to maximize the density of M'''

    while True:

        #If can't keep removing items, just terminate the function
        if len(S)==1 or len(T)==1: 
            break

        if S is not None:
            #Extract the row index of the row of minimal sum on indices in T
            up = S[np.argmin([rowsum(M, sp, T) for sp in S])]
        if T is not None:
            #Extract the column index of the column of minimal sum on indices in T
            vp = T[np.argmin([colsum(M, S, tp) for tp in T])]

        #Remove the indices of minimal row/column from their respective lists:
        S1 = S.copy().remove(up)
        T1 = T.copy().remove(vp)

        d0 = density(M,S,T)
        d1 = density(M,S1,T)
        d2 = density(M,S,T1)

        #If the densities increase after removing the row in S, do it:
        if d0<d1 and d2<d1:
            S.remove(up)
        #If the densities increase after removing the column in T, do it:
        elif d0<d2 and d1<=d2:
            T.remove(vp)
        #When the densities do not increase anymore, terminate
        else: break

    return S, T

In [5]:
class AnoComplete:
    def __init__(self, nr:int, nb:int, decay=0.9, tw=50):
        self.nameAlg = 'AnoComplete'
        self.time = 1
        self.nr = nr
        self.nb = nb
        self.decay = decay
        self.param = np.random.randint(1,1<<16,2*nr).astype(int)
        self.matrix = np.zeros((nb,nb),int)
        self.tw = tw # time window
        
        #Inital randomly picked 1x1 submatirces:
        self.Scur = [np.random.randint(nb)]
        self.Tcur = [np.random.randint(nb)]
    
    
    def call(self, u:int, v:int, t:int) -> float:
        ''''''
        
        if self.time < t:
            self.matrix = self.matrix*self.decay
            self.time = t
        u = ((347 * u) * self.param[0] + self.param[self.nr]) % self.nb
        v = ((347 * v) * self.param[0] + self.param[self.nr]) % self.nb
        self.matrix[u][v]+=1
        
        if self.name_alg == 'AnoEdgeLocal':
            self.Scur,self.Tcur = expand(self.matrix, self.Scur, self.Tcur, u, v)
            self.Scur,self.Tcur = condense(self.matrix, self.Scur, self.Tcur)
            return self.likelihood(self.matrix,self.Scur,self.Tcur,u,v)
        
        else:
            return self.EdgeSubDensity(self.matrix, u, v)

        
    def EdgeSubDensity(self, M, u:int, v:int) -> float:
        ''''''

        assert self.name_alg in ['AnoEdgeGlobal', 'AnoGraph'], \
        "The versions 'AnoEdgeLocal' and 'AnoGraph' do not use that method"

        S, Srem, T, Trem = list(range(self.nb)), list(range(self.nb)), list(range(self.nb)), list(range(self.nb))
        Scur, Tcur = [u], [v]
        Srem = Srem.remove(u)
        Trem = Trem.remove(v)
        dmax = density(M, Scur, Tcur)

        Stmp, Ttmp = Scur, Tcur # to be printed

        if self.name_alg == 'AnoGraph':
            while Srem or Trem:
                if Srem: up = S[np.argmax([rowsum(M,sp,Tcur) for sp in Srem])]
                if Trem: vp = T[np.argmax([colsum(M,Scur,tp) for tp in Trem])]
                if rowsum(M,up,Tcur)>colsum(M,Scur,vp):
                    Scur.append(up); Srem.remove(up)
                else:
                    Tcur.append(vp); Trem.remove(vp)
                dmax = max(dmax, density(M,Scur,Tcur))

                if dmax == density(M,Scur,Tcur):
                    Stmp,Ttmp = Scur,Tcur

        return dmax,Stmp,Ttmp

In [6]:
class AnoEdgeGlobal(AnoComplete):
    def __init__(self, nr:int, nb:int, decay=0.9, tw=50):
        super().__init__()
        self.name_alg = 'AnoEdgeGlobal'
             
    
class AnoEdgeLocal(AnoComplete):
    def __init__(self, nr:int, nb:int, decay=0.9, tw=50):
        super().__init__()
        self.nameAlg = 'AnoEdgeLocal'
    
    def likelihood(self, M, S, T, u:int, v:int) -> float:
        numerator = 0
        denominator = 0
        for s in S:
            numerator+=M[s][v]
            denominator+=1
        for t in T:
            numerator+=M[u][t]
            denominator+=1
        return numerator/denominator
    
    
    
class AnoGraph(AnoComplete):
    def __init__(self, nr:int, nb:int, decay=0.5, tw=50):
        super().__init__()
        self.nameAlg = 'AnoGraph'
        
        
    def AnoGraphDensity(self, M, u:int, v:int) -> float:
        Scur, Tcur = list(range(self.nb)), list(range(self.nb))
        dmax = density(M, Scur, Tcur)
        
        while Scur or Tcur:
            if Scur: up = Scur[np.argmin([rowsum(M,sp,Tcur) for sp in Scur])]
            if Tcur: vp = Tcur[np.argmin([colsum(M,Scur,tp) for tp in Tcur])]
            if rowsum(M,up,Tcur)<colsum(M,Scur,vp):
                Scur.remove(up)
            else:
                Tcur.remove(vp)
            dmax = max(dmax, density(M,Scur,Tcur))
        return dmax
    
    
class AnoGraph(AnoComplete):
    def __init__(self, nr:int, nb:int, decay=0.5, tw=50):
        super().__init__()
        self.nameAlg = 'AnoGraph'
    
    def AnographKDensity(self,M,k)->float:
        S = list(range(self.nb)); T = list(range(self.nb))
        dmax = 0
        for i in range(k):
            up,vp = divmod(np.argmax([M[s][t] for s in S for t in T]), len(S))
            dmax = max(dmax,self.EdgeSubDensity(M,up,vp))
        return dmax